In [1]:
import torch.optim as optim
import torch.nn.functional as F
import torch.nn as nn
import torch
torch.manual_seed(1)

# Linear Regression

In [3]:
x_train = torch.FloatTensor([[1], [2], [3], [4]])
y_train = torch.FloatTensor([[3], [5], [7], [9]])

W = torch.zeros(1, requires_grad=True)
b = torch.zeros(1, requires_grad=True)
#!
optimizer = torch.optim.SGD([W, b], lr=0.01)

epochs = 1000
for epoch in range(1, epochs+1):
    h = x_train*W+b
    cost = torch.mean((h-y_train)**2)

    optimizer.zero_grad()  # * gradient를 0으로 초기화
    cost.backward()   # * backpropagation을 통해 gradient계산
    optimizer.step()  # * 계산된 방향대로 W,b를 갱신(gradient descent)
print(cost, W, b)

tensor(3.4321e-05, grad_fn=<MeanBackward0>) tensor([2.0049], requires_grad=True) tensor([0.9857], requires_grad=True)


# mutlvariable

In [4]:
x_train = torch.FloatTensor([])

x1_train = torch.FloatTensor([[73], [93], [89], [96], [73]])
x2_train = torch.FloatTensor([[80], [88], [91], [98], [66]])
x3_train = torch.FloatTensor([[75], [93], [90], [100], [70]])
y_train = torch.FloatTensor([[152], [185], [180], [196], [142]])
x_train = torch.cat([x1_train, x2_train, x3_train], dim=1)
W = torch.zeros((3, 1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)
print(x_train.shape)

optimizer = optim.SGD([W, b], lr=1e-5)
epochs = 1000
for i in range(epochs):
    h = x_train.matmul(W) + b
    cost = torch.mean((h-y_train)**2)
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
print(W, b)

torch.Size([5, 3])
tensor([[0.7179],
        [0.6126],
        [0.6801]], requires_grad=True) tensor([0.0092], requires_grad=True)


# nn.Module (high-level implementation)

In [9]:
class MultivariateLinearRegressionModel(nn.Module):
    def __init__(self):
        super().__init__()    # nn.Module(부모 클래스)의 클래스 변수들을 가져올 수 있다.
        self.linear = nn.Linear(3,1)    #입력 차원과 출력 차원을 알려줌
    def forward(self, x):
        result = self.linear(x)
        return result

In [17]:
model = MultivariateLinearRegressionModel() #모델 초기화 (W,b따로 정의해줄 필요 없이)
optimizer = optim.SGD(model.parameters(),lr = 1e-6)
n_epochs = 200
for i in range(n_epochs):
    prediction = model(x_train)
    cost = F.mse_loss(prediction, y_train)
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    print('epoch: {:1d}/{} Cost: {:.6f}'.format(i+1, n_epochs, cost.item()))

epoch: 1/200 Cost: 11517.784180
epoch: 2/200 Cost: 10526.354492
epoch: 3/200 Cost: 9620.277344
epoch: 4/200 Cost: 8792.205078
epoch: 5/200 Cost: 8035.421875
epoch: 6/200 Cost: 7343.790039
epoch: 7/200 Cost: 6711.700195
epoch: 8/200 Cost: 6134.025879
epoch: 9/200 Cost: 5606.085449
epoch: 10/200 Cost: 5123.594727
epoch: 11/200 Cost: 4682.642578
epoch: 12/200 Cost: 4279.651367
epoch: 13/200 Cost: 3911.353516
epoch: 14/200 Cost: 3574.762939
epoch: 15/200 Cost: 3267.149658
epoch: 16/200 Cost: 2986.018066
epoch: 17/200 Cost: 2729.089355
epoch: 18/200 Cost: 2494.279297
epoch: 19/200 Cost: 2279.684326
epoch: 20/200 Cost: 2083.564209
epoch: 21/200 Cost: 1904.327881
epoch: 22/200 Cost: 1740.522705
epoch: 23/200 Cost: 1590.818359
epoch: 24/200 Cost: 1454.002441
epoch: 25/200 Cost: 1328.965088
epoch: 26/200 Cost: 1214.692871
epoch: 27/200 Cost: 1110.257690
epoch: 28/200 Cost: 1014.813843
epoch: 29/200 Cost: 927.586060
epoch: 30/200 Cost: 847.868347
epoch: 31/200 Cost: 775.013062
epoch: 32/200 Cost

# DATALOAD

https://pytorch.org/tutorials/beginner/data_loading_tutorial.html#iterating-through-the-dataset

In [19]:
from torch.utils.data import Dataset, DataLoader

In [21]:
class CustomDataset(Dataset):
    def __init__(self,csv_file = None):
        #데이터 로드(pandas로 csv파일 불러오는 등)
        self.x_data = [[73,80,75],[93,88,93],[96,98,100],[73,66,70]]
        self.y_data = [[152],[185],[196],[142]]
    def __len__(self):
        return len(self.x_data)
    def __getitem__(self,idx):
        x = torch.FloatTensor(self.x_data[idx])
        y = torch.FloatTensor(self.y_data[idx])
        return x,y
dataset = CustomDataset() 
dataloader = DataLoader(dataset,batch_size = 2, shuffle = True)
print(dataloader)

In [23]:
np_epochs = 20
for epoch in range(np_epochs+1):
    for batch_idx, samples in enumerate(dataloader):
        x_train,y_train = samples
        prediction = model(x_train)
        cost = F.mse_loss(prediction, y_train)
        optimizer.zero_grad()
        cost.backward()
        optimizer.step()
        print("epoch {:4d}/{} Batch{}/{} Cost: {:.6f}".format(epoch, np_epochs, batch_idx+1, len(dataloader),cost.item()))
        

epoch    0/20 Batch1/2 Cost: 3.504112
epoch    0/20 Batch2/2 Cost: 0.503981
epoch    1/20 Batch1/2 Cost: 0.519529
epoch    1/20 Batch2/2 Cost: 3.505044
epoch    2/20 Batch1/2 Cost: 3.448957
epoch    2/20 Batch2/2 Cost: 0.575886
epoch    3/20 Batch1/2 Cost: 0.466791
epoch    3/20 Batch2/2 Cost: 3.540088
epoch    4/20 Batch1/2 Cost: 0.715349
epoch    4/20 Batch2/2 Cost: 3.312751
epoch    5/20 Batch1/2 Cost: 0.525308
epoch    5/20 Batch2/2 Cost: 3.497788
epoch    6/20 Batch1/2 Cost: 3.496427
epoch    6/20 Batch2/2 Cost: 0.509208
epoch    7/20 Batch1/2 Cost: 3.250491
epoch    7/20 Batch2/2 Cost: 0.776163
epoch    8/20 Batch1/2 Cost: 0.523577
epoch    8/20 Batch2/2 Cost: 3.498270
epoch    9/20 Batch1/2 Cost: 3.442507
epoch    9/20 Batch2/2 Cost: 0.579620
epoch   10/20 Batch1/2 Cost: 0.523065
epoch   10/20 Batch2/2 Cost: 3.497982
epoch   11/20 Batch1/2 Cost: 3.496222
epoch   11/20 Batch2/2 Cost: 0.507411
epoch   12/20 Batch1/2 Cost: 0.469784
epoch   12/20 Batch2/2 Cost: 3.533459
epoch   13/2